In [1]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import MNIST, STL10
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
import tarfile
from torch import nn
import shutil
from tqdm import tqdm
from advbench.scripts.train import PD_ALGORITHMS

In [2]:
from pathlib import Path
import os
import wandb
import pandas as pd
import matplotlib.colors as mcolors
import os
import sys
import seaborn as sns
sys.path.append('../')
import numpy as np
import pandas as pd
from advbench.datasets import to_loaders
from advbench.algorithms import ERM, Augmentation, Adversarial_Worst_Of_K, Adversarial_PGD
from advbench.attacks import Fo_Adam
from advbench import hparams_registry
from advbench.lib import  misc
import torch
from advbench.lib.transformations import se_matrix, angle_to_rotation_matrix, se_transform
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn

import torch
import random
import numpy as np
import os
import json
import pandas as pd
import time
from humanfriendly import format_timespan


from advbench import datasets
from advbench import algorithms
from advbench import attacks
from advbench import hparams_registry
from advbench.lib import misc, meters, plotting, logging
from torch.cuda.amp import autocast
from torchsummary import summary
DEVICE = "cuda:1"
DOWNLOAD_WEIGHTS = True
PULL_METRICS = True

In [3]:
def load_weights(weight_path, dataset, device=DEVICE, hparams={}):       
        kw_args = {"perturbation": hparams["perturbation"]}
        if hparams["algorithm"] in PD_ALGORITHMS: 
            if hparams["algorithm"].endswith("Reverse"):
                kw_args["init"] = 0.0
            else:
                kw_args["init"] = 1.0
        print(hparams["perturbation"])
        algorithm = vars(algorithms)[hparams["algorithm"]](
        dataset.INPUT_SHAPE, 
        dataset.NUM_CLASSES,
        hparams,
        device,
        **kw_args).to(device)
        w_dict = torch.load(weight_path,map_location=DEVICE)
        #print(weight_dict.keys())
        #if db=="mnist":
        weight_dict = {}
        for s,v in w_dict.items():
                if "classifier" in s and "model" not in s:
                        weight_dict[s.replace('classifier', 'classifier.model')] = v
                else:
                        weight_dict[s] = v 
        algorithm.load_state_dict(weight_dict)
        return algorithm

In [4]:
if PULL_METRICS:
    api = wandb.Api(timeout=50)
    dsets = ["modelnet40"]#["STL10", "CIFAR100"]
    #results_dfs = {}
    entity = "hounie"
    all_runs_list = []
    for dataset in dsets:
        print("Fetching data for {}".format(dataset))
        project = f"OOD-{dataset}"
        base = f"../trained_weights/{project}"
        Path( base ).mkdir( parents=True, exist_ok=True )
        runs = api.runs(f"{entity}/{project}")
        print(f"{len(runs)} runs found")
        for run in tqdm(runs):
            if run.state == "finished" or (dataset=="modelnet40" and run.state != "running"):
                id = run.id
                run = api.run(f"{entity}/{project}/{id}")
                tmp_path = os.path.join(base, f"tmp")
                weight_dir = os.path.join(base, run.name)
                path = os.path.join(weight_dir, f"{run.id}.pkl")
                Path( weight_dir ).mkdir( parents=True, exist_ok=True )
                results = {**run.summary, **run.config, "weights": path, "id": run.id, "name":run.name}
                if DOWNLOAD_WEIGHTS:
                    try:
                        print(f"train-output/{run.name[:-4]} DGCNN {run.name[-1]}_ckpt.pkl")
                        f = run.file(f"train-output/{run.name[:-4]} DGCNN {run.name[-1]}_ckpt.pkl").download(tmp_path, replace=True) 
                        os.rename(f.name, path)
                        all_runs_list.append(results)
                    except:
                        print(f"{run.name} {run.id} failed")
                        
    df = pd.DataFrame(all_runs_list)
    df.to_csv(f"./results.csv")

Fetching data for modelnet40
59 runs found


  0%|                                                                                       | 0/59 [00:00<?, ?it/s]

train-output/finalPointcloudJitter Beta_PD_Reverse DGCNN 3_ckpt.pkl


  5%|████                                                                           | 3/59 [00:00<00:12,  4.43it/s]

train-output/finalPointcloudJitter Beta_PD_Reverse DGCNN 3_ckpt.pkl


...

 97%|███████████████████████████████████████████████████████████████████████████▎  | 57/59 [00:39<00:01,  1.62it/s]

train-output/finalPointcloudJitter Laplacian_DALE_PD_Reverse DGCNN 0_ckpt.pkl


 98%|████████████████████████████████████████████████████████████████████████████▋ | 58/59 [00:39<00:00,  1.46it/s]

train-output/finalPointcloudJitter Adversarial_Smoothed DGCNN 0_ckpt.pkl


100%|██████████████████████████████████████████████████████████████████████████████| 59/59 [00:40<00:00,  1.45it/s]


In [5]:
df = pd.read_csv("./results.csv")
test_keys = []
train_keys = []
for c in df.columns:
        if "loss" not in c and "acc" not in c:
            if c.startswith("test"):
                test_keys.append(c)
            else:
                train_keys.append(c)
    

In [7]:
from tqdm import tqdm
CLEAN_METRICS = False
data_dir = '../advbench/data'
device = DEVICE
failed = []
n_aug = {"Fo_PGD": 1}
t_attacks = [k for k in n_aug.keys()]
for exp_id in tqdm(df["id"]):
        if True:
            api = wandb.Api(timeout=50)
            exp = df[df["id"]==exp_id]
            dataset =  exp["dataset"].values[0]
            perturbation = exp["perturbation"].values[0]
            print(perturbation)
            model = exp["model"].values[0]
            if True:
                project = f"OOD-{dataset}"
                t_hparams = exp[test_keys].to_dict(orient='index')[exp.index.values[0]]
                test_hparams = {}
                for k, v in t_hparams.items():
                    test_hparams[k.replace("test_","")] = v
                print(exp.perturbation.values[0])
                if exp.perturbation.values[0] == 'SE':
                    hparams['epsilon'] = torch.tensor([hparams[f'epsilon_{i}'] for i in ("rot","tx","ty")]).to(device)
                    test_hparams['epsilon'] = torch.tensor([test_hparams[f'epsilon_{tfm}'] for tfm in ("rot","tx","ty")]).to(device)
                elif exp.perturbation.values[0] =='PointcloudTranslation':
                    hparams['epsilon'] = torch.tensor([hparams['epsilon_tx'] for i in range(3)] + [hparams['epsilon_ty'] for i in range(3)]).to(device)
                    test_hparams['epsilon'] = torch.tensor([test_hparams['epsilon_tx'] for i in range(3)] + [test_hparams['epsilon_ty'] for i in range(3)]).to(device)

                hparams = exp[train_keys].to_dict(orient='index')[exp.index.values[0]]
                hparams['model'] = model
                test_hparams['epsilon'] = hparams['epsilon']
                print(hparams['epsilon'])
                hparams['batched'] = True
                test_hparams['batched'] = True
                if "augment" in hparams:
                    aug = hparams["augment"]
                else:
                    aug = True
                dataset = vars(datasets)[dataset](data_dir, augmentation = aug)
                train_ldr, val_ldr, test_ldr = datasets.to_loaders(dataset, hparams, device=device)
                algorithm = load_weights(exp["weights"].values[0], dataset, device=DEVICE, hparams=hparams)           
                test_attacks = {a: vars(attacks)[a](algorithm.classifier, test_hparams, DEVICE, perturbation=hparams["perturbation"]) for a in t_attacks}
                wandb.init(project=project, resume=exp_id)
                if CLEAN_METRICS:
                    train_clean_acc, train_clean_mean_acc, train_clean_loss = misc.accuracy_mean_overall_loss(algorithm, train_ldr, device, max_batches=100)
                    val_clean_acc, val_clean_mean_acc, val_clean_loss = misc.accuracy_mean_overall_loss(algorithm, val_ldr, device)
                    test_clean_acc, test_clean_mean_acc, test_clean_loss = misc.accuracy_mean_overall_loss(algorithm, test_ldr, device)
                    wandb.log({'train_clean_loss': train_clean_loss,'train_clean_acc': train_clean_acc, 'train_clean_acc_bal': train_clean_mean_acc})
                    wandb.log({'best_val_clean_loss': val_clean_loss, 'best_val_clean_acc': val_clean_acc, 'best_val_clean_acc_bal': val_clean_mean_acc})
                    wandb.log({'test_clean_loss': test_clean_loss,'test_clean_acc': test_clean_acc, 'test_clean_acc_bal': test_clean_mean_acc})
                for attack_name, attack in test_attacks.items():
                    print(f"Logging {attack_name}...")
                    test_adv_acc, test_adv_acc_mean, test_adv_acc_bal, test_adv_acc_mean_bal, adv_loss, accs, loss, deltas = misc.adv_accuracy_loss_delta_balanced(algorithm, test_ldr, device, attack, augs_per_batch=1)
                    print("Test Adversarial Accuracy:", test_adv_acc)
                    print("Test Balanced Adversarial Accuracy:", test_adv_acc_bal)
                    train_adv_acc, train_adv_acc_mean, train_adv_acc_bal, train_adv_acc_mean_bal, adv_loss, train_accs, train_loss, train_deltas = misc.adv_accuracy_loss_delta_balanced(algorithm, val_ldr, device, attack, augs_per_batch=1)
                    print("Train Adversarial Accuracy:", train_adv_acc)
                    print("Train Balanced Adversarial Accuracy:", train_adv_acc_bal)
                    wandb.log({'test_acc_adv_'+attack_name: test_adv_acc,'test_acc_adv_mean_'+attack_name: test_adv_acc_mean, 'test_loss_adv_'+attack_name: loss.mean(),
                     'test_acc_adv_bal_'+attack_name: test_adv_acc_bal, 'test_acc_adv_mean_bal_'+attack_name: test_adv_acc_mean_bal})
                    wandb.log({'train_acc_adv_'+attack_name: train_adv_acc,'train_acc_adv_mean_'+attack_name: train_adv_acc_mean, 'train_loss_adv_'+attack_name: loss.mean(),
                     'train_acc_adv_bal_'+attack_name: train_adv_acc_bal, 'train_acc_adv_mean_bal_'+attack_name: train_adv_acc_mean_bal})                                                   
                wandb.finish(quiet=True)
        else:
            failed.append(exp_id)

with open('failed.txt', 'w') as f:
    for line in failed:
        f.write(f"{line}\n")

  0%|                                                                                       | 0/51 [00:00<?, ?it/s]

PointcloudJitter
PointcloudJitter
0.05
PointcloudJitter
model DGCNN


best_val_clean_acc,93.29268
best_val_clean_acc_bal,90.94158
best_val_clean_loss,1.40777
clean loss_avg,1.27758
delta L1-border_avg,-0.00286
dual variable_avg,0.02312
epoch,249
loss_avg,1.30795
robust loss_avg,1.31352
step,2214000
test_clean_acc,91.57212


Logging Fo_PGD...



0it [00:00, ?it/s]
1it [00:04,  4.12s/it]
2it [00:07,  3.76s/it]
3it [00:11,  3.65s/it]
4it [00:14,  3.60s/it]
5it [00:18,  3.57s/it]
6it [00:21,  3.55s/it]
7it [00:25,  3.55s/it]
8it [00:28,  3.54s/it]
9it [00:32,  3.54s/it]
10it [00:35,  3.54s/it]
11it [00:39,  3.54s/it]
12it [00:42,  3.54s/it]
13it [00:46,  3.54s/it]
14it [00:50,  3.55s/it]
15it [00:53,  3.55s/it]
16it [00:57,  3.55s/it]
17it [01:02,  3.67s/it]
  0%|                                                                                       | 0/51 [01:09<?, ?it/s]


KeyboardInterrupt: 